In [3]:
import pandas as pd
from minio import Minio
from minio.error import S3Error
import io

# Configurações do Minio
minio_host = 'minio.ifpbapps.online'
minio_access_key = "VQdTNNVXZZLXNMRnV4RDFoQ2MtZGFSRjN3Q0Z2Z"
minio_secret_key = "pMnQ4ZkktaU9UdlRYX1c!3S1lHMkVpajU5LTRLTHl1NwbmRDVmdiWW9RaGJvZmFWSkF5YmZ"
minio_bucket = 'raios'

# Conectar ao Minio
minio_client = Minio(minio_host, access_key=minio_access_key, secret_key=minio_secret_key, secure=True)

# Função para baixar arquivo do Minio e carregar como DataFrame
def download_and_read_csv(minio_client, bucket, file_path):
    try:
        # Baixar o arquivo do Minio
        data = minio_client.get_object(bucket, file_path)
        
        # Ler o CSV
        df = pd.read_csv(data)
        
        return df
    except S3Error as err:
        print(err)
        return None

# Lista de arquivos no Minio
files = [
    'Consolidado Raios 2023_EPB.csv',
    'Consolidado Raios 2023_ERO.csv',
    'Consolidado Raios 2023_EMT.csv',
    'Consolidado Raios 2023_EMS.csv',
    'Consolidado Raios 2023_EMR.csv',
    'Consolidado Raios 2023_EAC.csv',
    'Consolidado Raios 2023_ETO.csv',
    'Consolidado Raios 2023_ESS.csv',
    'Consolidado Raios 2023_ESE.csv'
]

# Lista para armazenar DataFrames
dfs = []

# Iterar sobre os arquivos e baixar do Minio
for file in files:
    df = download_and_read_csv(minio_client, minio_bucket, file)
    if df is not None:
        dfs.append(df)

# Concatenar os DataFrames
Descargas_Energisa = pd.concat(dfs, ignore_index=True)

In [2]:
minio_secure = True

# Nome do bucket e pasta no Minio
minio_bucket = 'raios'
minio_folder = ''

# Inicializa o cliente Minio
minio_client = Minio(minio_host, access_key=minio_access_key, secret_key=minio_secret_key, secure=True)

# Carrega os códigos do IBGE
cod_ibge_data = minio_client.get_object(minio_bucket, f'{minio_folder}dados-ibge.xls')
cod_ibge_buffer = io.BytesIO(cod_ibge_data.read())
cod_ibge = pd.read_excel(cod_ibge_buffer, header=6)

colunas_ocorrencias = ['DscOcorrenciaAberta', 'CodIBGE']

# Carrega as ocorrências da ANEEL
ocorrencias_data = minio_client.get_object(minio_bucket, f'{minio_folder}ocorrencias-emergenciais-rede-distribuicao-2023.csv')
ocorrencias_buffer = io.BytesIO(ocorrencias_data.read())
ocorrencias = pd.read_csv(ocorrencias_buffer, delimiter=';', encoding='latin-1', low_memory=False, usecols=colunas_ocorrencias)

# Se chegou até aqui, os dados foram carregados com sucesso
print("Dados carregados com sucesso!")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
Dados carregados com sucesso!


In [ ]:
# Deixando um único dataframe pra todas as descargas do grupo Energisa
Lista_Descargas = [Descargas_EPB, Descargas_ERO, Descargas_EMT, Descargas_EMS, Descargas_EMR, Descargas_EAC, Descargas_ETO, Descargas_ESS, Descargas_ESE]

# Mantendo apenas as colunas da base que possuem dados de raios no sentido Nuvem-Solo
Descargas_Energisa = Descargas_Energisa.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]

#  Excluir a primeira linha, pois a tabela só começa na terceira linha
Descargas_Energisa = Descargas_Energisa.drop(0)

# Renomear todas as colunas para que fique de acordo com o número de raios nos meses correspondentes 
novos_nomes_colunas = ['Localidade', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
Descargas_Energisa.columns = novos_nomes_colunas

# Filtrando apenas ocorrências que sejam advindas de descargas atmosféricas
Ocorrencias = ocorrencias[ocorrencias['DscOcorrenciaAberta'] == "INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;DESCARGA ATMOSFERICA"]

# Agora, adicionando a coluna "Cod_IBGE" para cada DataFrame na Lista_Descargas
for base in Lista_Descargas:
    # Obtém o estado da base atual
    estado = base.iloc[0]['Localidade']
    
    # Filtra o DataFrame Cod_IBGE para o estado correspondente
    cod_ibge_estado = Cod_IBGE[Cod_IBGE['Nome_UF'] == estado]
    
    # Mescla os DataFrames com base no município
    base = pd.merge(base, cod_ibge_estado[['Nome_Município', 'Cod_IBGE']], how='left', left_on='Localidade', right_on='Município')
    
    # Remove a coluna redundante
    base = base.drop(columns='Município')
    
    # Renomeia a coluna Cod_IBGE
    base = base.rename(columns={'Cod_IBGE': 'Cod_IBGE'})

# Agora, cada DataFrame na Lista_Descargas terá uma nova coluna 'Cod_IBGE'

In [4]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 7.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 13.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=0511c443cf6b7528c6a690dbae90831cce8efc4152fad330d4c1b28fa40cf032
  Stored in directory: /home/ruanv/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.
